<a id="top"></a>
# Pneumonia Classification Demo

## Check for latest version
<br><div class=danger><b>Important: Before proceeding, please run the following cell to ensure that you are running the most recent version of this sample.</b></div>

In [ ]:
# Show the current status of this and all documents with ability to update
from qarpo.catalog import DemoCatalog
import os
status = DemoCatalog(os.getcwd(), "Demo").ShowRepositoryControls()

## Introduction

This demo shows how a smart video IoT solution may be created using Intel® hardware and software tools to perform pneumonia classification.  This solution uses an inference model that has been trained to classify the presence of pneumonia using a patient's chest X-ray.  The results are visualized from what the network has learned using the Class Activation Maps (CAM) technique.

### Key concepts
This demo includes an example for the following:
- Application:
  - Image input is supported using OpenCV
  - Visualization of the resulting class activation maps
- Intel® DevCloud for the Edge:
  - Submitting inference as jobs that are performed on different edge compute nodes (rather than on the development node hosting this Jupyter* notebook)
  - Monitoring job status
  - Viewing results and assessing performance for hardware on different compute nodes
- [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit):
  - Create the necessary Intermediate Representation (IR) files for the inference model using the [Model Downloader](http://docs.openvinotoolkit.org/latest/_tools_downloader_README.html) and [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html)
  - Run an inference application on multiple hardware devices using the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html)
  - Access and use of a class activation map as part of inference

### Application background 
Pneumonia is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli.
Typically symptoms include some combination of cough, chest pain, fever, and trouble breathing [1].
Pneumonia affects approximately 450 million people globally (7% of the population) and results in about 4 million deaths per year [2]. To diagnose this disease, chest X-ray images remain the best diagnostic tool.

![](example-pneumonia.jpeg) *Chest X-ray image of a patient with Pneumonia*

In this application, we use a model trained to classify patients with pneumonia over healthy cases based on their chest X-ray images. The topology used is the DenseNet 121 which is an architecture that has shown to be very efficient at this problem.  DenseNet 121 is the first work to claim a classification rate better than practicing radiologists. The dataset used for training is from the "Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification" [3] with a [CC BY 4.0 license](https://creativecommons.org/licenses/by/4.0/).
The trained model is provided as a frozen Tensorflow model.

[1] Ashby B, Turkington C (2007). The encyclopedia of infectious diseases (3rd ed.). New York: Facts on File. p. 242. ISBN 978-0-8160-6397-0. 

[2] Ruuskanen O, Lahti E, Jennings LC, Murdoch DR (April 2011). "Viral pneumonia". Lancet. 377 (9773): 1264–75. 

[3] [Labeled Optical Coherence Tomography (OCT) and Chest X-Ray Images for Classification](https://data.mendeley.com/datasets/rscbjbr9sj/2) [(Direct link to dataset)](http://dx.doi.org/10.17632/rscbjbr9sj.2#file-41d542e7-7f91-47f6-9ff2-dd8e5a5a7861)

### Classification and class activation maps

A Class Activation Map (CAM) [1] is a technique to visualize the regions that are relevant within a Convolutional Neural Network to identify the specific class in the image. 
The CAM $M(x,y)$ is calculated from the *N* feature maps $f_i(x,y)$ within the last convolutional layer of the network.  The weighted sum of those feature maps based on the weights of the fully connected layer $w_i$, represents how important those feature maps are for the given classification output. 

$ M(x,y)=\sum_{i=0}^N w_i f_i(x,y) $

When using the Intel® Distribution of OpenVINO™ toolkit, this can be implemented using the Python* code (Found in [classification_pneumonia.py](./classification_pneumonia.py)):

```python
def class_activation_map_openvino(res, convb, fc, net, fp16):
    res_bn = res[convb]
    conv_outputs=res_bn[0,:,:,:]
    # retrieve layer weights
    weights_fc=net.layers.get(fc).blobs["weights"]
    # initialize CAM array
    cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[1:])
    # perform weighted sum
    for i, w in enumerate(weights_fc):
        conv_outputs1=conv_outputs[i, :, :]
        if fp16:
            w=float16_conversion(w)
            conv_outputs1=float16_conversion_array(conv_outputs[i, :, :])
        cam += w * conv_outputs1
    return cam
```

To calculate the Class Activation Map, we access the output feature maps of the last convolution layer `convb` and the weights of the fully connected layer `fc`. 
By default, only the output layer is present in the returned output of the network. In order to get the feature maps, the last convolution layer must be added to the output. 
This is done by using the function [`add_outputs(`*convb*`)`](https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IENetwork.html) on the network before loading it to the the device plugin. 
In order to obtain the layer's name, we recommend [Netron](https://pypi.org/project/netron/), which allows to visualize graphically the model. 

In our pneumonia classification model, the name of the last convolutional layer `convb` is "relu_1/Relu" and the name of the fully-connected layer `fc` is "predictions_1/MatMul".


In order to get the feature maps, inference must be performed first to get the results. The `res` argument in the code above is the inference output which includes the feature maps by using `add_outputs()`.
We access the `fc` layer weights using the call [`net.layers.get(fc).blobs["weights"]`](https://docs.openvinotoolkit.org/latest/ie_python_api/classie__api_1_1IENetLayer.html) on the network `net`.  Before doing the weighted sum, if the FP16 model is being used, the weights and feature maps value must be converted from type `float16`to type `float` using the function `float16_conversion_array()`. The weighted sum of the weights is then done with the feature maps. 

The result of the function is an image of same size as the feature maps (here, $7\times7$).  An example is shown below on the left.
The CAM image is upsampled to the original input image size and overlaid over the input image. The region of highest value (here, in yellow) will be the region that is the most relevant to deciding on the classification result value. 

![](example_CAM.png) *CAM image and overlay of CAM with input image*

[1] Zhou, Bolei, et al. "Learning deep features for discriminative localization." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

## Prerequisites

### Install Pillow
The Python* source code depends upon the ['Pillow`](https://pypi.org/project/Pillow/) module which is not present by default and must be installed before running this sample.

Run the following cell to test for and install `Pillow`.

In [ ]:
try:
    # Test whether Pillow package is installed
    import PIL
    print("Pillow already installed")
except:
    # Exception while importing Pillow, try to install it using pip3
    print("Installing Pillow")
    !pip3 install Pillow

## Pneumonia classification application
The pneumonia classification application uses the Intel® Distribution of OpenVINO™ toolkit to perform inference on an input X-ray image to classify the presence of pneumonia.  We will setup, run, and view the results for this application for several different hardware devices (CPU. GPU, etc.) available on the compute nodes within the Intel® DevCloud for the Edge.  To accomplish this, we will be performing the following tasks:

1. Use the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) to create the inference model IR files needed to perform inference
2. Create the job file used to submit running inference on compute nodes
3. Submit jobs for different compute nodes and monitor the job status until complete
4. View results and assess performance 

### How it works
At startup the pneumonia classification application configures itself by parsing the command line arguments.  Once configured, the application loads the specified inference model's IR files into the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) and runs inference on the specified input X-ray images.  The result for each input X-ray image is written to the `results.txt` file in the form:
   Pneumonia probability: <*% probability*>, Inference performed in <*time*>, Input file: <*input filename*>

To run the application on the Intel® DevCloud for the Edge, a job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and and Intel® Arria® 10 FPGA.  After inference on the input is completed, the output is stored in the appropriate `results/<architecture>/` directory.  The results are then viewed within this Jupyter* Notebook using the `show_results` utility.

The application and inference code for this sample is already implemented in the Python* files: [`classification_pneumonia.py`](./classification_pneumonia.py), [`utils.py`](./utils.py), and [`utils_image.py`](./utils_image.py).

The following sections will guide you through configuring and running the pneumonia classification demo.

### Create the IR files for the inference model

The Intel® Distribution of OpenVINO™ toolkit includes the [Model Optimizer](http://docs.openvinotoolkit.org/latest/_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) used to convert and optimize trained models into the Intermediate Representation (IR) model files, and the [Inference Engine](http://docs.openvinotoolkit.org/latest/_docs_IE_DG_Deep_Learning_Inference_Engine_DevGuide.html) that uses the IR model files to run inference on hardware devices.  The IR model files can be created from trained models from popular frameworks (e.g. Caffe\*, Tensorflow*, etc.).  For this sample, the model is supplied in the file `model.pb` 

The `model.pb` file will need to be optimized using the Model Optimizer to create the necessary IR files.  We will be running the inference model on different hardware devices which have different requirements on the precision of the model (see [Inference Engine Supported Model Formats](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html#supported_model_formats) for details).  For our purposes, we will focus on the use of the two most common precisions, FP32 and FP16.

For this model, we will run the Model Optimizer using the format:
```bash
/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model <path_to_model> \
    --input_shape=[<N,H,W,C>] \
    --data_type <data_precision> \
    -o <path_to_output_directory> \
    --mean_values [<channel_mean_values>] \
    --scale_values [<scale_values>]
```

The input arguments are as follows:
- **--input_model** : The model's input *.pb* file
- **--input_shape** : The model's input data shape
 - **N** = Batch size
 - **H** = Height
 - **W** = Width
 - **C** = Number of channels
- **--data_type** : The model's data type and precision (e.g. FP16, FP32, INT8, etc.)
- **--o** : Output directory where to store the generated IR model files
- **--scale_values** : Scaling (divide by, one per channel) value to apply to input values
- **--mean_values** : Mean values (one per channel) to be subtracted from input values before scaling

The input shape, scaling values, and mean values we will be using are specific to the model topology being used.  Using the appropriate values for the model that we will use, The complete command will look like the following:
```bash
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
    --input_model model/crnn.pb \
    --input_shape=[1,224,224,3] \
    --data_type <data_precision> \
    -o models/<data_precision> \
    --mean_values [123.75,116.28,103.58] \
    --scale_values [58.395,57.12,57.375]
```
We will run the command twice, first with <*data_precision*> set to `FP16` and then `FP32` to get all the IR files we will need to run inference on different devices.

<br><div class=note><i><b>Note: </b>More information on how to use Model Optimizer to convert TensorFlow* models as well as specifying input shape and scaling parameters for common model topologies may be found at:[Converting a TensorFlow* Model](https://docs.openvinotoolkit.org/latest/_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html)
</i></div>


Run the following cell to use the Model Optimizer to create the `FP16` and `FP32` model IR files:

In [ ]:
# Create FP16 IR files
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model model.pb \
--input_shape=[1,224,224,3] \
--data_type FP16 \
-o models/FP16/ \
--mean_values [123.75,116.28,103.58] \
--scale_values [58.395,57.12,57.375]

# Create FP32 IR files
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model model.pb \
--input_shape=[1,224,224,3] \
--data_type FP32 \
-o models/FP32 \
--mean_values [123.75,116.28,103.58] \
--scale_values [58.395,57.12,57.375] 

# find all resulting IR files
!echo "\nAll IR files that were downloaded or created:"
!find ./models -name "*.xml" -o -name "*.bin"

<br><div class=tip><i><b>Tip: </b>The '!' at the beginning of a line is a special Jupyter* Notebook command that allows you to run shell commands as if you are at a command line. The above command will also work in a terminal (with the '!' removed).</i></div>

As shown above from the output of the last `!find...` command, the required sets of IR model files (`*.xml` and `*.bin`) have been created.  

### Optional exercise: View input images without inference

If you are curious to see the input images, run the following cell to view the original video stream used for inference and this sample.

In [ ]:
import glob
from IPython.display import Image, display
InputImages = 'validation_images/PNEUMONIA/*.jpeg'
# find all the input images
files=glob.glob(InputImages)
# display input images
for image_file in files:
    print(f"Input image file {image_file}:")
    display(Image(image_file))


### Run the demo

Running the next cell will display an interactive user interface allowing you to submit jobs to run the demo on multiple edge compute nodes selected by hardware devices, view the output of each job, and compare performance results across jobs.

To run a job:
1. Select the desired option in the **Target node** list
2. Select the desired device in the **Target architecture** list
3. Click the **Submit** button

After the **Submit** button is clicked, a tab will appear for the new job with a label in the format "*status*: *JobID*".  Once the job status appears as "Done", the **Display output** button may be clicked to view the output for the job.

After one or more jobs are done, the performance results for each job may be plotted by clicking the **Plot results** button.  Results for each job will be potted as bar graphs for **inference time** and **frames per second**.  Lower values mean better performance for **inference time** and higher values mean better performance for **frames per second**. When comparing results, please keep in mind that some architectures are optimized for highest performance, others for low power or other metrics.

Run the next cell to begin the interactive demo.

In [ ]:
# import necessary libraries
import json
import qarpo

# load job configurations for demo
with open('job_config.json') as json_file:  
    data = json.load(json_file)
    
# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

## Next steps
- The complete [Pneumonia Classification Sample](../../developer_samples/pneumonia-classification/classification_pneumonia.ipynb) for this demo
- [Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) -  tutorials on using and creating Jupyter* Notebooks
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>